<a href="https://colab.research.google.com/github/tomek-l/fire/blob/master/5.resnet_binary_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline

import torch
import torchvision
import torchvision.transforms as transforms

In [0]:
import torchvision.models as models

class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        self.backbone = models.resnet18(pretrained=True)
        self.backbone.fc = torch.nn.Linear(512,1)
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.sigmoid(x)

        return x

In [3]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [4]:
!ls '/gdrive/My Drive/cmpe_lab276/FireVideo/binary_dataset/train'

negatives  positives


In [0]:
def load_dataset():


  tr = torchvision.transforms.Compose([torchvision.transforms.Resize((224,224)),
                                  torchvision.transforms.ToTensor()])

  train_dataset = torchvision.datasets.ImageFolder(root='/gdrive/My Drive/cmpe_lab276/FireVideo/binary_dataset/train',
                                                  transform=tr)

  train_loader = torch.utils.data.DataLoader(
      train_dataset,
      batch_size=16,
      num_workers=0,
      shuffle=True
  )

  return train_loader

In [0]:
trainloader = load_dataset()
batch_list = list(trainloader)

In [0]:
def accuracy(pred, truth):
    agreeing = (pred.transpose(0,1)[0] >= 0.5).eq(truth >= 0.5)
    acc = float(agreeing.sum())/agreeing.numel()
    return acc

In [12]:
import torch.optim as optim

model = Model()
device = torch.device("cuda:0")
model = model.to(device)

criterion = torch.nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5, weight_decay=0)

for epoch in range(30):  # epochs

    running_loss = 0.0
    running_acc = 0.0

    for i, data in enumerate(batch_list):
        # get the inputs; data is a list of [inputs, labels]
        inputs = data[0].to(device)
        labels = data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs[:,0], labels.type_as(outputs[:,0]))
        loss.backward()
        optimizer.step()

        acc = accuracy(outputs, labels)
        # print statistics
        
        running_loss += loss.item()
        running_acc += acc
        if i%20 == 19:
          print(f'epoch: {epoch+1}, batch: {i}, loss: {running_loss/20}, accuracy: {running_acc/20}')
          running_loss = 0.0
          running_acc = 0.0

print('Finished Training')

epoch: 1, batch: 19, loss: 0.6570004180073739, accuracy: 0.584375
epoch: 1, batch: 39, loss: 0.48078336715698244, accuracy: 0.76875
epoch: 1, batch: 59, loss: 0.3219021797180176, accuracy: 0.93125
epoch: 1, batch: 79, loss: 0.2940948836505413, accuracy: 0.921875
epoch: 1, batch: 99, loss: 0.20993741378188133, accuracy: 0.94375
epoch: 2, batch: 19, loss: 0.18122894316911697, accuracy: 0.946875
epoch: 2, batch: 39, loss: 0.14054579045623541, accuracy: 0.96875
epoch: 2, batch: 59, loss: 0.1588105084374547, accuracy: 0.959375
epoch: 2, batch: 79, loss: 0.14356687851250172, accuracy: 0.96875
epoch: 2, batch: 99, loss: 0.09171762261539698, accuracy: 0.990625
epoch: 3, batch: 19, loss: 0.11616989932954311, accuracy: 0.971875
epoch: 3, batch: 39, loss: 0.08137623704969883, accuracy: 0.984375
epoch: 3, batch: 59, loss: 0.11304593272507191, accuracy: 0.9625
epoch: 3, batch: 79, loss: 0.09040649197995662, accuracy: 0.978125
epoch: 3, batch: 99, loss: 0.05152659909799695, accuracy: 0.996875
epoch: